In [1]:
import json
import pandas as pd
import os

PROFILING_DIR = '../consegna/profilings/GPT-4o'
INPUT_DIR = '../consegna/LLMAGENTS_CLAIMS'

Load profilings csv into dataframes

In [2]:
value_profilings = pd.read_csv(os.path.join(PROFILING_DIR, 'VALUE_PROFILING.csv'))
value_profilings = value_profilings.map(lambda x: x.lower() if isinstance(x, str) else x)
print(value_profilings)

AttributeError: 'DataFrame' object has no attribute 'map'

Load claim and convert it to dataframes

In [2]:
def load_json_as_df(filename):
    with open(os.path.join(INPUT_DIR, filename), 'r') as f:
        json_data = json.load(f)
    rows = []
    for key, entry in json_data.items():
        row = {}
        row['claim_id'] = key
        # Extract specifications
        for spec_key, spec in entry["specifications"].items():
            row[spec["name"]] = spec["value"]
        # Add measure and outcome
        row["Measure"] = entry["Measure"]
        row["Outcome"] = entry["Outcome"]
        rows.append(row)
    claim_df = pd.DataFrame(rows)
    filename = filename.replace("_claims.json","")
    claim_df['paper_id'] = filename.split('_')[0]
    claim_df['table_id'] = filename.split('_')[1]
    return claim_df

reassign values in dataframe

In [3]:
claims_df = []
for filename in os.listdir(INPUT_DIR):
    df = load_json_as_df(filename)
    print(df)
    claims_df.append(df)


   claim_id                     Dataset           Task    Method Measure  \
0         0  HaluEval Li et al. (2023a)             QA  HaluEval    Acc.   
1         1  HaluEval Li et al. (2023a)             QA  HaluEval       R   
2         2  HaluEval Li et al. (2023a)             QA  HaluEval       P   
3         3  HaluEval Li et al. (2023a)             QA  HaluEval      F1   
4         4  HaluEval Li et al. (2023a)             QA   FACTOOL    Acc.   
5         5  HaluEval Li et al. (2023a)             QA   FACTOOL       R   
6         6  HaluEval Li et al. (2023a)             QA   FACTOOL       P   
7         7  HaluEval Li et al. (2023a)             QA   FACTOOL      F1   
8         8  HaluEval Li et al. (2023a)             QA      Ours    Acc.   
9         9  HaluEval Li et al. (2023a)             QA      Ours       R   
10       10  HaluEval Li et al. (2023a)             QA      Ours       P   
11       11  HaluEval Li et al. (2023a)             QA      Ours      F1   
12       12 

In [11]:
with open('alignment_name_schema.json', 'r') as file:
    align_name = json.load(file)

with open('alignment_values_schema.json', 'r') as file:
    align_value = json.load(file)

aligned_df = pd.DataFrame(columns=align_name.keys())

excluded_columns = ['outcome', 'table_id', 'paper_id', 'claim_id']

for claim_df in claims_df:
    for _, row in claim_df.iterrows():
        aligned_row = {key: None for key in align_name.keys()}
        aligned_row["outcome"] = row["Outcome"]
        aligned_row["table_id"] = row["table_id"]
        aligned_row["paper_id"] = row["paper_id"]
        aligned_row["claim_id"] = row["claim_id"]
        for name_key, name_values in align_name.items():
            for col in [c for c in claim_df.columns if c not in excluded_columns]:
                if row[col].lower() in name_values:
                    for value_key, value_value in align_value.items():
                        if value_value is not None and row[col].lower() in value_value:
                            aligned_row[name_key] = value_key
                            print(f"{aligned_row["paper_id"]}_{aligned_row["table_id"]}_{aligned_row["claim_id"]} value: {row[col]}")
                            break
                        else:
                            aligned_row[name_key] = row[col]
        aligned_df = pd.concat([aligned_df, pd.DataFrame([aligned_row])], ignore_index=True)

aligned_df.to_csv('result.csv')

2406.03075_3_0 value: HaluEval Li et al. (2023a)
2406.03075_3_0 value: Acc.
2406.03075_3_1 value: HaluEval Li et al. (2023a)
2406.03075_3_1 value: R
2406.03075_3_2 value: HaluEval Li et al. (2023a)
2406.03075_3_3 value: HaluEval Li et al. (2023a)
2406.03075_3_3 value: F1
2406.03075_3_4 value: HaluEval Li et al. (2023a)
2406.03075_3_4 value: Acc.
2406.03075_3_5 value: HaluEval Li et al. (2023a)
2406.03075_3_5 value: R
2406.03075_3_6 value: HaluEval Li et al. (2023a)
2406.03075_3_7 value: HaluEval Li et al. (2023a)
2406.03075_3_7 value: F1
2406.03075_3_8 value: HaluEval Li et al. (2023a)
2406.03075_3_8 value: Acc.
2406.03075_3_9 value: HaluEval Li et al. (2023a)
2406.03075_3_9 value: R
2406.03075_3_10 value: HaluEval Li et al. (2023a)
2406.03075_3_11 value: HaluEval Li et al. (2023a)
2406.03075_3_11 value: F1
2406.03075_3_12 value: HaluEval Li et al. (2023a)
2406.03075_3_12 value: Acc.
2406.03075_3_13 value: HaluEval Li et al. (2023a)
2406.03075_3_13 value: R
2406.03075_3_14 value: HaluE

JSON pe paulo  

In [5]:
def retrieve_spec_id(filename, claim_id, flag):
    with open(os.path.join(INPUT_DIR, filename), 'r') as f:
        data = json.load(f)
    
    for key, value in data[claim_id]['specifications'].items():
        if value['value'] == flag:
            return key

In [6]:
def constr_filename(paper_id, table_id):
    return f'{paper_id}_{table_id}_claims.json'

In [40]:
def build_json_name(df):
    json_data = {}
    name2spec = {}
    for col in df.columns:
        if col not in ['claim_id', 'table_id', 'paper_id', 'Outcome']:
            nomi = []
            for _,row in df.iterrows():
                filename = constr_filename(row['paper_id'], row['table_id'])
                spec_id = retrieve_spec_id(filename, row['claim_id'], row[col])
                if (spec_id is not None):
                    nomi.append(f"{row['paper_id']}_{row['table_id']}_{row['claim_id']}_{spec_id}")  
            name2spec[col] = nomi
        json_data['aligned_names'] = name2spec
    return json_data

json_data = build_json_name(aligned_df)

In [49]:
import re
files = """ 
2406.03075_3_0 HaluEval Li et al. (2023a)
2406.03075_3_0 Acc.
2406.03075_3_1 HaluEval Li et al. (2023a)
2406.03075_3_1 R
2406.03075_3_2 HaluEval Li et al. (2023a)
2406.03075_3_3 HaluEval Li et al. (2023a)
2406.03075_3_3 F1
2406.03075_3_4 HaluEval Li et al. (2023a)
2406.03075_3_4 Acc.
2406.03075_3_5 HaluEval Li et al. (2023a)
2406.03075_3_5 R
2406.03075_3_6 HaluEval Li et al. (2023a)
2406.03075_3_7 HaluEval Li et al. (2023a)
2406.03075_3_7 F1
2406.03075_3_8 HaluEval Li et al. (2023a)
2406.03075_3_8 Acc.
2406.03075_3_9 HaluEval Li et al. (2023a)
2406.03075_3_9 R
2406.03075_3_10 HaluEval Li et al. (2023a)
2406.03075_3_11 HaluEval Li et al. (2023a)
2406.03075_3_11 F1
2406.03075_3_12 HaluEval Li et al. (2023a)
2406.03075_3_12 Acc.
2406.03075_3_13 HaluEval Li et al. (2023a)
2406.03075_3_13 R
2406.03075_3_14 HaluEval Li et al. (2023a)
2406.03075_3_15 HaluEval Li et al. (2023a)
2406.03075_3_15 F1
2406.03075_3_16 HaluEval Li et al. (2023a)
2406.03075_3_16 Acc.
2406.03075_3_17 HaluEval Li et al. (2023a)
2406.03075_3_17 R
2406.03075_3_18 HaluEval Li et al. (2023a)
2406.03075_3_19 HaluEval Li et al. (2023a)
2406.03075_3_19 F1
2406.03075_3_20 HaluEval Li et al. (2023a)
2406.03075_3_20 Acc.
2406.03075_3_21 HaluEval Li et al. (2023a)
2406.03075_3_21 R
2406.03075_3_22 HaluEval Li et al. (2023a)
2406.03075_3_23 HaluEval Li et al. (2023a)
2406.03075_3_23 F1
2406.03075_3_24 HaluEval Li et al. (2023a)
2406.03075_3_24 Acc.
2406.03075_3_25 HaluEval Li et al. (2023a)
2406.03075_3_25 R
2406.03075_3_26 HaluEval Li et al. (2023a)
2406.03075_3_27 HaluEval Li et al. (2023a)
2406.03075_3_27 F1
2406.03075_3_28 HaluEval Li et al. (2023a)
2406.03075_3_28 Acc.
2406.03075_3_29 HaluEval Li et al. (2023a)
2406.03075_3_29 R
2406.03075_3_30 HaluEval Li et al. (2023a)
2406.03075_3_31 HaluEval Li et al. (2023a)
2406.03075_3_31 F1
2406.03075_3_32 HaluEval Li et al. (2023a)
2406.03075_3_32 Acc.
2406.03075_3_33 HaluEval Li et al. (2023a)
2406.03075_3_33 R
2406.03075_3_34 HaluEval Li et al. (2023a)
2406.03075_3_35 HaluEval Li et al. (2023a)
2406.03075_3_35 F1
2402.10890_3_12 CodeLlama-13B (E)
2402.10890_3_13 CodeLlama-13B (E)
2402.10890_3_14 CodeLlama-13B (E)
2402.10890_3_15 CodeLlama-13B (E)
2402.10890_3_16 GPT-3.5-Turbo (E)
2402.10890_3_17 GPT-3.5-Turbo (E)
2402.10890_3_18 GPT-3.5-Turbo (E)
2402.10890_3_19 GPT-3.5-Turbo (E)
2402.10890_3_20 CodeLlama-13B-FT (E)
2402.10890_3_21 CodeLlama-13B-FT (E)
2402.10890_3_22 CodeLlama-13B-FT (E)
2402.10890_3_23 CodeLlama-13B-FT (E)
2407.17115_5_18 Role-playing+
2407.17115_5_19 Role-playing+
2407.17115_5_20 Role-playing+
2407.17115_5_21 Role-playing+
2407.17115_5_22 Role-playing+
2407.17115_5_23 Role-playing+
2407.17115_5_24 Role-playing+
2407.17115_5_25 Role-playing+
2407.17115_5_26 Role-playing+
2407.17115_5_27 History records+
2407.17115_5_28 History records+
2407.17115_5_29 History records+
2407.17115_5_30 History records+
2407.17115_5_31 History records+
2407.17115_5_32 History records+
2407.17115_5_33 History records+
2407.17115_5_34 History records+
2407.17115_5_35 History records+
2407.17115_5_45 Output format+
2407.17115_5_46 Output format+
2407.17115_5_47 Output format+
2407.17115_5_48 Output format+
2407.17115_5_49 Output format+
2407.17115_5_50 Output format+
2407.17115_5_51 Output format+
2407.17115_5_52 Output format+
2407.17115_5_53 Output format+
2407.17115_5_54 RPP+
2407.17115_5_55 RPP+
2407.17115_5_56 RPP+
2407.17115_5_57 RPP+
2407.17115_5_58 RPP+
2407.17115_5_59 RPP+
2407.17115_5_60 RPP+
2407.17115_5_61 RPP+
2407.17115_5_62 RPP+
2406.12707_1_0 BERTScore
2406.12707_1_2 BERTScore
2406.12707_1_4 BERTScore
2402.10890_1_0 Acc
2402.10890_1_1 F1
2402.10890_1_4 Acc
2402.10890_1_5 F1
2402.10890_1_8 Acc
2402.10890_1_9 F1
2402.10890_1_12 Acc
2402.10890_1_13 F1
2402.10890_1_16 Acc
2402.10890_1_17 F1
2402.10890_1_20 Acc
2402.10890_1_21 F1
2402.10890_1_24 Acc
2402.10890_1_25 F1
2402.10890_1_28 Acc
2402.10890_1_29 F1
2402.10890_1_32 Acc
2402.10890_1_33 F1
2402.10890_1_36 Acc
2402.10890_1_37 F1
2402.10890_1_40 Acc
2402.10890_1_41 F1
2402.10890_1_44 Acc
2402.10890_1_45 F1
2402.10890_1_48 Acc
2402.10890_1_49 F1
2402.10890_1_52 Acc
2402.10890_1_53 F1
2402.10890_1_56 Acc
2402.10890_1_57 F1
2402.10890_1_60 Acc
2402.10890_1_61 F1
2402.10890_1_64 Acc
2402.10890_1_65 F1
2402.10890_1_68 Acc
2402.10890_1_69 F1
2405.17129_1_6 ZSEC-gpt4o
2405.17129_1_7 ZSEC-gpt4o
2405.17129_1_8 ZSEC-gpt4o
2406.03075_4_0 HaluEval Li et al. (2023a)
2406.03075_4_0 Acc.
2406.03075_4_1 HaluEval Li et al. (2023a)
2406.03075_4_1 R
2406.03075_4_2 HaluEval Li et al. (2023a)
2406.03075_4_3 HaluEval Li et al. (2023a)
2406.03075_4_3 F1
2406.03075_4_4 HaluEval Li et al. (2023a)
2406.03075_4_4 Acc.
2406.03075_4_5 HaluEval Li et al. (2023a)
2406.03075_4_5 R
2406.03075_4_6 HaluEval Li et al. (2023a)
2406.03075_4_7 HaluEval Li et al. (2023a)
2406.03075_4_7 F1
2406.03075_4_8 HaluEval Li et al. (2023a)
2406.03075_4_8 Acc.
2406.03075_4_9 HaluEval Li et al. (2023a)
2406.03075_4_9 R
2406.03075_4_10 HaluEval Li et al. (2023a)
2406.03075_4_11 HaluEval Li et al. (2023a)
2406.03075_4_11 F1
2406.03075_4_12 HaluEval Li et al. (2023a)
2406.03075_4_12 Acc.
2406.03075_4_13 HaluEval Li et al. (2023a)
2406.03075_4_13 R
2406.03075_4_14 HaluEval Li et al. (2023a)
2406.03075_4_15 HaluEval Li et al. (2023a)
2406.03075_4_15 F1
2406.03075_2_0 Factool Chern et al. (2023)
2406.03075_2_0 Self-Check (0)
2406.03075_2_0 Acc.
2406.03075_2_1 Factool Chern et al. (2023)
2406.03075_2_1 Self-Check (0)
2406.03075_2_1 R
2406.03075_2_2 Factool Chern et al. (2023)
2406.03075_2_2 Self-Check (0)
2406.03075_2_3 Factool Chern et al. (2023)
2406.03075_2_3 Self-Check (0)
2406.03075_2_3 F1
2406.03075_2_4 Factool Chern et al. (2023)
2406.03075_2_4 Self-Check (3)
2406.03075_2_4 Acc.
2406.03075_2_5 Factool Chern et al. (2023)
2406.03075_2_5 Self-Check (3)
2406.03075_2_5 R
2406.03075_2_6 Factool Chern et al. (2023)
2406.03075_2_6 Self-Check (3)
2406.03075_2_7 Factool Chern et al. (2023)
2406.03075_2_7 Self-Check (3)
2406.03075_2_7 F1
2406.03075_2_8 Factool Chern et al. (2023)
2406.03075_2_8 Acc.
2406.03075_2_9 Factool Chern et al. (2023)
2406.03075_2_9 R
2406.03075_2_10 Factool Chern et al. (2023)
2406.03075_2_11 Factool Chern et al. (2023)
2406.03075_2_11 F1
2406.03075_2_12 Factool Chern et al. (2023)
2406.03075_2_12 Acc.
2406.03075_2_13 Factool Chern et al. (2023)
2406.03075_2_13 R
2406.03075_2_14 Factool Chern et al. (2023)
2406.03075_2_15 Factool Chern et al. (2023)
2406.03075_2_15 F1
2406.03075_2_16 Factool Chern et al. (2023)
2406.03075_2_16 Self-Check (0)
2406.03075_2_16 Acc.
2406.03075_2_17 Factool Chern et al. (2023)
2406.03075_2_17 Self-Check (0)
2406.03075_2_17 R
2406.03075_2_18 Factool Chern et al. (2023)
2406.03075_2_18 Self-Check (0)
2406.03075_2_19 Factool Chern et al. (2023)
2406.03075_2_19 Self-Check (0)
2406.03075_2_19 F1
2406.03075_2_20 Factool Chern et al. (2023)
2406.03075_2_20 Self-Check (3)
2406.03075_2_20 Acc.
2406.03075_2_21 Factool Chern et al. (2023)
2406.03075_2_21 Self-Check (3)
2406.03075_2_21 R
2406.03075_2_22 Factool Chern et al. (2023)
2406.03075_2_22 Self-Check (3)
2406.03075_2_23 Factool Chern et al. (2023)
2406.03075_2_23 Self-Check (3)
2406.03075_2_23 F1
2406.03075_2_24 Factool Chern et al. (2023)
2406.03075_2_24 Acc.
2406.03075_2_25 Factool Chern et al. (2023)
2406.03075_2_25 R
2406.03075_2_26 Factool Chern et al. (2023)
2406.03075_2_27 Factool Chern et al. (2023)
2406.03075_2_27 F1
2406.03075_2_28 Factool Chern et al. (2023)
2406.03075_2_28 Acc.
2406.03075_2_29 Factool Chern et al. (2023)
2406.03075_2_29 R
2406.03075_2_30 Factool Chern et al. (2023)
2406.03075_2_31 Factool Chern et al. (2023)
2406.03075_2_31 F1
2406.12707_2_0 BERTScore
2406.12707_2_2 BERTScore
2406.12707_2_4 BERTScore
2405.17129_5_3 Acc.
2405.17129_5_7 Acc.
2405.17129_5_11 Acc.
2405.17129_5_15 Acc.
2310.01444_2_0 HotpotQA
2310.01444_2_0 CoT [61]
2310.01444_2_1 HotpotQA
2310.01444_2_2 HotpotQA
2310.01444_2_3 HotpotQA
2310.01444_2_4 HotpotQA
2310.01444_2_5 HotpotQA
2310.01444_2_6 HotpotQA
2310.01444_2_7 HotpotQA
2310.01444_2_8 HotpotQA
2310.01444_2_9 HotpotQA
2310.01444_2_10 HotpotQA
"""
value2filename = {}
for line in files.split('\n'):
    match = re.match(r"^(\S+)\s(.+)$", line)
    if match:
        base, value = match.groups()
        filename = constr_filename(base.split('_')[0], base.split('_')[1])
        spec_id = retrieve_spec_id(filename, base.split('_')[2], value)
        if value.lower().strip() not in value2filename.keys():
            value2filename[value.lower()] = []
        if spec_id is not None:
            value2filename[value.lower()].append(f"{base}_{spec_id}")
        else:
            value2filename[value.lower()].append(f"{base}_measure")

value2filename_good = {}
for key_1, value_1 in value2filename.items():
    for key_2, value_2 in align_value.items():
        if value_2 is not None and key_1 in value_2:
            value2filename_good[key_2] = value_1

json_data['aligned_values'] = value2filename_good

In [50]:
with open('task3.json', 'w') as f:
    json.dump(json_data, f, indent=2)

Values alignment

In [37]:
def extract_values_from_json(json_data):
    values_set = set()

    def extract_values(obj):
        if isinstance(obj, dict):
            for value in obj.values():
                extract_values(value)
        elif isinstance(obj, list):
            for item in obj:
                extract_values(item)
        else:
            values_set.add(obj)

    extract_values(json_data)
    return values_set

# Load the JSON file
with open('alignment_schema.json', 'r') as file:
    json_data = json.load(file)

# Extract values
values_set = extract_values_from_json(json_data)
sorted_list = sorted(list(values_set))
output_dict = {key: None for key in sorted_list}


output_file = "alignment_values_schema.json"
with open(output_file, "w") as json_file:
    json.dump(output_dict, json_file, indent=4)